#Lab.06 / IBM3202 – Acoplamiento Molecular con AutoDock

###Aspectos Teóricos

A medida que se determinan experimentalmente más y más estructuras de proteínas utilizando cristalografía de rayos X o espectroscopía de resonancia magnética nuclear (RMN), el acoplamiento molecular se utiliza cada vez más como una herramienta en **el descubrimiento de medicamentos**.

Las simulaciones de acoplamiento molecular exploran las posiciones potenciales de unión de pequeñas moléculas en el **sitio de unión** de una proteína objetivo para la cual se dispone de una estructura determinada experimentalmente, ya sea mediante cristalografía de rayos X o RMN. El acoplamiento contra blancos proteicos generados por **modelado comparativo** también se vuelve posible para proteínas cuyas estructuras aún no se han resuelto.

De esta manera, la **_farmacabilidad_** de diferentes compuestos y su afinidad de unión en un blanco proteico dado pueden calcularse para procesos posteriores de optimización de leads.


<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/docking_01.png'/>
<figcaption>FIGURA 1. En el acoplamiento molecular, la unión se evalúa en dos pasos: A) Energética de la transición de los estados no unidos del ligando y el objetivo hacia las conformaciones del complejo unido; y B) energética de la unión proteína-ligando en estas conformaciones.
 <br> Huey R et al (2007) <i>J Comput Chem 28(6), 1145-1152.</i></figcaption></center>
</figure>

Los programas de acoplamiento molecular realizan un **algoritmo de búsqueda** en el que se evalúan de manera recursiva las conformaciones variables de un ligando dado, generalmente generadas mediante algoritmos Monte Carlo o genéticos, hasta alcanzar la convergencia a un mínimo de energía. Finalmente, a través de una **función de puntuación de afinidad**, se estima y emplea un ΔG [energía libre de unión en kcal/mol] para clasificar las poses candidatas como la suma de varias contribuciones energéticas (electrostáticas, van der Waals, desolvatación, etc.).


##Generalidades experimentales

Normalmente, en esta sesión de laboratorio acoplamos un análogo dímero del polímero plástico PET (2PET) en el sitio de unión de la PET hidrolasa de *I. sakaiensis*, una enzima degradadora de PET descubierta en Japón en 2016 para la cual nuestro laboratorio resolvió su estructura tridimensional [Fecker T et al (2018) *Biophys J 114 (6), 1302-1312*].

Sin embargo, inspirados por la pandemia de COVID-19, en esta sesión de laboratorio realizaremos un ensayo de acoplamiento de **indinavir**, un componente activo de la **terapia antirretroviral para tratar el VIH**, en el sitio de unión de su proteína objetivo, **la proteasa del VIH-2**.

Para nuestra sesión de laboratorio, instalaremos y utilizaremos **MGLtools** (y alternativamente **pdb2pqr**) para preparar los archivos de la proteína objetivo, **OpenBabel** para preparar los archivos del ligando, **AutoDock Vina** para el procedimiento de acoplamiento y **py3Dmol** para establecer la configuración adecuada de la cuadrícula de búsqueda y analizar los resultados.


<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/docking_02.png' />
<figcaption>FIGURA 2. Pasos generales del acoplamiento molecular. Primero, se parametriza la proteína objetivo y el ligando o ligandos. Luego, se prepara el sistema configurando la cuadrícula de búsqueda. Una vez que se realiza el cálculo de acoplamiento, las poses del ligando se puntúan según una función de energía dada. Por último, la búsqueda computacional se procesa y se compara con datos experimentales para su validación. <br><i>Tomado de Pars Silico (en.parssilico.com)</i></figcaption>
.</i></figcaption></center>
</figure>

#Parte 0 – Descargando e instalando el software necesario

Antes de comenzar, debes **recordar iniciar el tiempo de ejecución alojado en Google Colab**.

Luego, debemos instalar varios programas para realizar este tutorial. A saber:
- **biopython** para la manipulación de archivos PDB.
- **py3Dmol** para la visualización de la estructura de la proteína y la configuración de la cuadrícula de búsqueda.
- **miniconda**, un instalador mínimo gratuito de **conda** para la gestión de paquetes de software y entornos.
- **OpenBabel** para la parametrización de nuestro o nuestros ligandos.
- **MGLtools** para la parametrización de nuestra proteína objetivo utilizando cargas Gasteiger.
- **pdb2pqr** para la parametrización de nuestra proteína utilizando el campo de fuerza AMBER ff99.
- **AutoDock Vina** para el proceso de acoplamiento.

Después de varias pruebas, las siguientes instrucciones de instalación son la mejor manera de configurar **Google Colab** para esta sesión de laboratorio.

1. Primero instalaremos biopython, py3Dmol y pdb2pqr de la siguiente manera:


In [ ]:
#Instalando py3Dmol
!pip -q install py3Dmol
#Instalando biopython
!pip -q install biopython
#Instalando pdb2pqr v3.0
!pip -q install pdb2pqr
#Instalando RDkit
!pip -q install rdkit-pypi

In [ ]:
#Importar py3Dmol
import py3Dmol

In [ ]:
#Revisando si pdb2pqr se instaló correctamente
!pdb2pqr30 -h | awk 'NR==1{if($1=="usage:") print "PDB2PQR succesfully installed"; else if($1!="usage:") print "Something went wrong. Please install again"}'

2. Y luego instalaremos conda para poder instalar MGLtools y OpenBabel:


In [ ]:
#Instalando condacolab
!pip install -q condacolab
import condacolab
condacolab.install_miniconda()

#Instalando MGLtools y OpenBabel desde bioconda
!conda install -c conda-forge -c bioconda mgltools openbabel zlib ncurses --yes

3. Finalmente, descargaremos el programa Autodock Vina desde [AutoDock Vina GitHub](https://github.com/ccsb-scripps/AutoDock-Vina) y crearemos un alias para usarlo durante esta sesión:


In [ ]:
#Download and extract Autodock Vina from SCRIPPS
#Then, we set up an alias for vina to be treated as a native binary
%%bash
mkdir vina
cd vina
wget -q https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_1.2.5_linux_x86_64
chmod +x /content/vina/vina_1.2.5_linux_x86_64
wget -q https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_split_1.2.5_linux_x86_64
chmod +x /content/vina/vina_split_1.2.5_linux_x86_64

In [ ]:
%alias vina /content/vina/vina_1.2.5_linux_x86_64
%alias vina_split /content/vina/vina_1.2.5_linux_x86_64

Una vez que se completen estos procesos de instalación de software, estamos listos para realizar nuestros experimentos.


# Parte 1 – Descarga y Preparación del Receptor para AutoDock


1. El primer paso en un procedimiento de acoplamiento molecular es tener una estructura de una proteína objetivo dada. Si bien en algunos casos se puede utilizar un modelo comparativo de alta calidad, la mayoría de los casos comienzan con una estructura tridimensional resuelta experimentalmente (rayos X, RMN, criomicroscopía electrónica).

   En este escenario, se descarga la estructura de una proteína objetivo desde el **Banco de Datos de Proteínas (PDB)** (https://www.rcsb.org/pdb) utilizando un ID de acceso específico. Por ejemplo, la PET hidrolasa resuelta por nuestro laboratorio tiene el ID de acceso 6ANE.

   Para este tutorial, utilizaremos la estructura de la proteasa del VIH-2, resuelta mediante cristalografía de rayos X y depositada en el PDB con el ID de acceso 1HSG, que descargaremos utilizando **biopython**:


In [ ]:
#Descargando los pdbs usando bio
import os
from Bio.PDB import *
pdbid = ['1hsg']
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")

2. En el caso de la difracción de rayos X, esta estrategia experimental no discrimina entre la densidad electrónica proveniente de átomos de proteínas estáticos o moléculas de agua. Esto significa que la mayoría de las estructuras de proteínas resueltas por difracción de rayos X también incluyen las llamadas **aguas cristalográficas** (ver los puntos rojos no enlazados en la estructura de proteínas a continuación). Estas moléculas no son importantes para nuestra simulación de acoplamiento en particular, y también debemos eliminarlas.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/docking_03.png' />
<figcaption>FIGURA 3. Representación esquemática del dímero de la proteasa del VIH-2 (ID de acceso al PDB 1HSG), con sus residuos de N a C terminales coloreados de azul a rojo en el espectro del arco iris, mostrando las aguas cristalográficas como esferas rojas.</i></figcaption></center>
</figure>

  Por lo general, esto se puede hacer fácilmente extrayendo todas las líneas del archivo PDB que comienzan con **"ATOM"**, ya que así se denominan todos los átomos que pertenecen a residuos de aminoácidos y nucleicos. En cambio, los átomos de ligandos, moléculas de agua y otros residuos no proteicos/no nucleicos suelen denominarse **"HETATM"**. Además, las diferentes cadenas de un oligómero están separadas por una cadena **"TER"**, que es importante mantener en nuestro caso.

El siguiente **script de Python** primero creará una carpeta en la que almacenaremos todos los datos relacionados con nuestro experimento de acoplamiento molecular. Luego, extraerá todas las líneas que coincidan con la cadena "ATOM" (para los átomos de proteína) o "TER" (para las separaciones de cadenas) en un archivo PDB separado para su procesamiento posterior. Por favor, revísalo detenidamente.


In [ ]:
# Este script creará una carpeta llamada "single-docking" para nuestro experimento.
# Luego, imprimirá todas las líneas "ATOM" y "TER" de un archivo PDB dado en un nuevo archivo.

# Creamos primero una carpeta. Necesitamos importar las bibliotecas os y path.
import os
from pathlib import Path

# Luego, definimos la ruta de la carpeta que queremos crear.
# Observa que la carpeta HOME para una ejecución alojada en Colab es /content/.
singlepath = Path("/content/single-dock/")

# Ahora, creamos la carpeta usando el comando os.mkdir().
# La condición if es solo para verificar si la carpeta ya existe,
# en cuyo caso Python devuelve un error.
if os.path.exists(singlepath):
  print("La ruta ya existe.")
if not os.path.exists(singlepath):
  os.mkdir(singlepath)
  print("La ruta se creó correctamente.")

# Ahora asignamos una variable "protein" con el nombre y la extensión de nuestro archivo PDB.
protein = "1hsg.pdb"

# Y utilizamos el siguiente script para imprimir selectivamente las líneas que contienen la
# cadena "ATOM" y "TER" en un nuevo archivo dentro de nuestra carpeta recién creada.
with open(singlepath / "1hsg_prot.pdb", "w") as g:
  f = open(protein, 'r')
  for line in f:
    row = line.split()
    if row[0] == "ATOM":
      g.write(line)
    elif row[0] == "TER":
      g.write("TER\n")
  g.write("END")
  print("Archivo creado con éxito.")


3. Una vez que hemos impreso las líneas "ATOM" del archivo PDB principal, tenemos un nuevo archivo que contiene las coordenadas de nuestra proteína objetivo.

   Sin embargo, para que AutoDock Vina realice un experimento de acoplamiento molecular, el archivo de la proteína objetivo debe contener tipos de átomos compatibles con Autodock para evaluar diferentes tipos de interacciones, así como sus cargas parciales para evaluar interacciones electrostáticas. Esta información se incluye en un archivo conocido como **PDBQT**, una modificación del formato PDB que también incluye **cargas (q)** y **tipos de átomos específicos de AutoDock (t)** en dos columnas adicionales al final del archivo PDBQT. Cabe destacar, sin embargo, que AutoDock Vina **ignora las cargas parciales proporcionadas por el usuario**, ya que tiene su propia forma de tratar con las interacciones electrostáticas.

   Por último, la proteína objetivo debe contener **todos los hidrógenos polares**, de modo que se puedan formar enlaces de hidrógeno entre la proteína objetivo y el ligando. La mayoría de las estructuras de proteínas no incluyen hidrógenos, lo que significa que debemos agregarlos.

   ¡Esta es la parte del tutorial donde tienes **dos opciones diferentes** para continuar con tu experimento!


3. a) Add the polar hydrogens of your protein and parameterize it with **Gasteiger** charges and atom types using **MGLtools** (this is the canonical option for the majority of AutoDock users)

In [ ]:
# Parametrización y adición de cargas Gasteiger en nuestra proteína usando MGLtools
!prepare_receptor4.py -r $singlepath/1hsg_prot.pdb -o $singlepath/1hsg_prot.pdbqt -A hydrogens -U nphs_lps -v

3. b) Agregar los hidrógenos polares de tu proteína y parametrizarla basándote en el pKa de cada aminoácido a pH 7.4 con el force field **AMBER99ff** utilizando **pdb2pqr**, seguido de la eliminación de los hidrógenos no polares y la conversión en un archivo **PDBQT** usando **MGLtools**.

   En este caso, pdb2pqr genera un archivo intermedio **PQR**, una modificación del formato PDB que permite a los usuarios agregar parámetros de carga y radio a los datos PDB existentes. Esta información no se altera durante el uso de **MGLtools**.


In [ ]:
# Primero, utilizando pdb2pqr para parametrizar nuestro receptor con AMBER99ff, manteniendo
# las identificaciones de cadena y configurando el receptor a un pH de 7.4
!pdb2pqr30 --ff AMBER --keep-chain --titration-state-method propka --with-ph 7.4 $singlepath/1hsg_prot.pdb $singlepath/1hsg_prot.pqr

# Luego, convertir el archivo .pqr en un archivo .pdbqt mientras se eliminan los hidrógenos no polares
# pero sin cambiar los parámetros AMBER agregados a la proteína
!prepare_receptor4.py -r $singlepath/1hsg_prot.pqr -o $singlepath/1hsg_prot.pdbqt -C -U nphs_lps -v


**¡Todo listo con tu proteína objetivo!**

> Antes de pasar a preparar el ligando para el acoplamiento molecular, ten en cuenta las siguientes preguntas:
- ¿Por qué es importante agregar hidrógenos para nuestros fines de simulaciones de acoplamiento?
- ¿Por qué solo agregamos hidrógenos polares?
- Si las cargas parciales son ignoradas por Autodock Vina, ¿cómo pueden afectar mis resultados de acoplamiento estas diferentes estrategias? (¡esto es algo que realmente puedes probar!)


#Parte 2 – Descargando y preparando el ligando para AutoDock

1. Ahora necesitamos preparar el ligando que usaremos para nuestro análisis de acoplamiento. En nuestro caso, usaremos **Indinavir**. Este medicamento es un inhibidor de la proteasa utilizado como componente de la terapia antirretroviral para tratar el VIH/SIDA, ayudando a disminuir la carga viral. En esta oportunidad, intentaremos predecir la posición de acoplamiento de indinavir en el sitio de unión de la proteasa del VIH-2.

   Primero, comenzaremos creando una carpeta en la que almacenaremos nuestros ligandos para el acoplamiento molecular.


In [ ]:
# Comencemos creando primero una carpeta. Necesitamos importar las bibliotecas os y path.
import os
from pathlib import Path

# Primero, crearemos una ruta para todos los ligandos que utilizaremos en este tutorial.
# Observa que la carpeta HOME para una ejecución alojada en Colab es /content/.
ligandpath = Path("/content/ligands/")

# Ahora, creamos la carpeta usando el comando os.mkdir().
# La condición if es solo para verificar si la carpeta ya existe,
# en cuyo caso Python devuelve un error.
if os.path.exists(ligandpath):
  print("La ruta de los ligandos ya existe.")
if not os.path.exists(ligandpath):
  os.mkdir(ligandpath)
  print("La ruta de los ligandos se creó correctamente.")


2. Ahora, descargaremos indinavir de la base de datos **DrugBank** (*Nucleic Acids Res
. 2006; 34, D668-72*). Esta es una base de datos en línea integral y de libre acceso que contiene información sobre medicamentos y blancos de medicamentos. Puedes consultar la información química, farmacológica y farmacéutica detallada sobre Indinavir [en este enlace de DrugBank](https://www.drugbank.ca/drugs/DB00224).

   Descargaremos este ligando en formato SMILES para continuar con su preparación para el acoplamiento molecular.


In [ ]:
#Descargando Indinavir del DrugBank en formato SMILES
!wget -q https://www.drugbank.ca/structures/small_molecule_drugs/DB00224.smiles -P $ligandpath

3. ¡Oye! Pero, ¿qué es el formato SMILES? Bueno, el **Simplified Molecular-Input Line-Entry System** (SMILES) es una notación de texto que permite a un usuario representar una estructura química de una manera que pueda ser utilizada por la computadora. La notación elemental para diferentes tipos de enlaces entre diferentes átomos es la siguiente:

  \-	para enlaces simples (por ejemplo, C-C o CC para CH3CH3)

  \=	para enlaces dobles (por ejemplo, C=C para CH2CH2)

  \#	para enlaces triples (por ejemplo, C#N para C≡N)

  \	para enlace aromático (por ejemplo, C\*1\*C\*C\*C\*C\*C1 o c1ccccc1 para benceno)

  \. para estructuras desconectadas (por ejemplo, Na.Cl para NaCl)

  / y \ para estereoisómeros de enlace doble (por ejemplo, F/C=C/F para trans-1,2-difluoroetileno y F/C=C\F para cis-1,2-difluoroetileno)

  @ y @@ para enantiómeros (por ejemplo, N\[C@@H](C)C(=O)O para L-alanina y N\[C@H](C)C(=O)O para D-alanina)

  **¡Echemos un vistazo al SMILES de Indinavir!**


In [ ]:
# Imprimir el SMILES de indinavir para ver de qué se trata
print((ligandpath / "DB00224.smiles").read_text())

In [ ]:
#@title Utiliza el siguiente visor para cargar tu SMILES como una molécula 3D

import py3Dmol
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

def MolTo3DView(mol, size=(300, 300), style="stick", surface=False, opacity=0.5):
    assert style in ('line', 'stick', 'sphere', 'carton')
    mblock = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view()
    viewer.addModel(mblock, 'mol')
    viewer.setStyle({style:{}})
    if surface:
        viewer.addSurface(py3Dmol.SAS, {'opacity': opacity})
    viewer.zoomTo()
    return viewer

from ipywidgets import interact,fixed,IntSlider
import ipywidgets

def smi2conf(smiles):
    '''Convert SMILES to rdkit.Mol with 3D coordinates'''
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol)
        AllChem.MMFFOptimizeMolecule(mol, maxIters=200)
        return mol
    else:
        return None

@interact
def smi2viewer(smi='CC=O'):
    try:
        conf = smi2conf(smi)
        return MolTo3DView(conf).show()
    except:
        return None

4. Ahora, tomaremos este formato SMILES y lo utilizaremos para construir y parametrizar una estructura tridimensional de Indinavir en formato **PDBQT** para su uso en el acoplamiento molecular. Al igual que con el receptor, también tenemos diferentes opciones para preparar nuestro ligando para el acoplamiento molecular:


4. a) Utiliza el programa **babel** para convertir el SMILES en un archivo **MOL2** sin ningún trabajo adicional (como buscar los mejores conformeros), excepto configurar el estado de protonación a pH 7.4, y luego utiliza **MGLtools** para parametrizar el ligando con cargas parciales de **Gasteiger** (esta es la opción canónica para la mayoría de los usuarios de AutoDock). Ten en cuenta que estamos generando un ligando en el que **todas las torsiones son activas** durante el procedimiento de acoplamiento.


In [ ]:
## Convirtiendo Indinavir de SMILES a un formato PDB tridimensional
!obabel $ligandpath/DB00224.smiles -O indinavir.mol2 --gen3d best -p 7.4 --canonical
# Parametrizando y agregando cargas de Gasteiger en nuestro ligando utilizando MGLtools
# Agregar -z lleva a un ligando rígido sin torsiones
!prepare_ligand4.py -l indinavir.mol2 -o $singlepath/indinavir.pdbqt -U nphs_lps -v
# NOTA: por alguna razón, MGLtools no reconoce el ligando cuando está en una carpeta diferente
# Aquí estamos eliminando el archivo PDB temporal requerido para generar el archivo PDBQT
os.remove("indinavir.mol2")

4. b) Utiliza el programa **babel** para convertir el SMILES en un archivo **MOL2** tridimensional mientras realiza una minimización de energía utilizando el Campo de Fuerza Generalizado Amber (**GAFF**). Luego, utiliza **MGLtools** para parametrizar el ligando con cargas parciales de **Gasteiger**. Ten en cuenta que estamos generando un ligando en el que **todas las torsiones son activas** durante el procedimiento de acoplamiento.


In [ ]:
# Convirtiendo Indinavir de SMILES a un formato MOL2 tridimensional y realizando una minimización de energía del conformero utilizando el campo de fuerza GAFF
# Luego, prepara el ligando para el acoplamiento utilizando el script de Autodock
!obabel $ligandpath/DB00224.smiles -O indinavir.mol2 --gen3d --best --canonical --minimize --ff GAFF --steps 10000 --sd
!prepare_ligand4.py -l indinavir.mol2 -o $singlepath/indinavir.pdbqt -U nphs_lps -v
os.remove("indinavir.mol2")


4. c) Utiliza el programa **babel** para convertir el SMILES en un archivo **MOL2** tridimensional mientras realiza una búsqueda ponderada de rotor para el conformero de menor energía utilizando el Campo de Fuerza Generalizado Amber (**GAFF**). Luego, utiliza **MGLtools** para parametrizar el ligando con cargas parciales de **Gasteiger**. Ten en cuenta que estamos generando un ligando en el que **todas las torsiones son activas** durante el procedimiento de acoplamiento.


In [ ]:
# Convirtiendo Indinavir de SMILES a un formato MOL2 tridimensional y realizando una búsqueda ponderada de rotor para el conformero de menor energía
# Luego, prepara el ligando para el acoplamiento utilizando el script de Autodock
!obabel $ligandpath/DB00224.smiles -O indinavir.mol2 --gen3d --best --canonical --conformers --weighted --nconf 50 --ff GAFF
!prepare_ligand4.py -l indinavir.mol2 -o $singlepath/indinavir.pdbqt -U nphs_lps -v
os.remove("indinavir.mol2")


**¡Ya está todo listo con tu ligando!** Ahora, pasamos a configurar el experimento de acoplamiento molecular.


# Parte 3 – Configuración y realización del acoplamiento molecular con AutoDock


1. Como se explicó en las clases, es necesario definir el espacio de búsqueda para el acoplamiento molecular en una proteína objetivo mediante el uso de una **caja de rejilla**. Esta caja de rejilla suele centrarse dentro del sitio de unión, sitio activo o alostérico de la proteína objetivo y su tamaño será lo suficientemente grande como para que **todas las residuos de unión estén dentro de la caja de rejilla**.

  Aquí, haremos uso de **py3Dmol** para inspeccionar visualmente la estructura de la proteína en representación de caricaturas y dibujar una caja de rejilla. La posición y el tamaño de la caja de rejilla se definirán mediante las coordenadas de su centroide y sus dimensiones en x, y z.

  Para guiar mejor la búsqueda de las dimensiones y coordenadas óptimas de la caja de rejilla, también mostraremos las residuos Val32, Ile47 y Val82 de la proteasa del VIH-2.

  El script que define el visualizador, que llamamos **ViewProtGrid**, se carga primero en **Colab** con las siguientes líneas de código.


In [ ]:
#@title Cargando el script que crea nuestro visor de la caja de acoplamiento
#These definitions will enable loading our protein and then
#drawing a box with a given size and centroid on the cartesian space
#This box will enable us to set up the system coordinates for the simulation
#
#HINT: The active site of the HIV-2 protease is near the beta strands in green
#
#ACKNOWLEDGE: This script is largely based on the one created by Jose Manuel
#Napoles Duarte, Physics Teacher at the Chemical Sciences Faculty of the
#Autonomous University of Chihuahua (https://github.com/napoles-uach)
#
#First, we define the grid box
def definegrid(object,bxi,byi,bzi,bxf,byf,bzf):
  object.addBox({'center':{'x':bxi,'y':byi,'z':bzi},'dimensions': {'w':bxf,'h':byf,'d':bzf},'color':'blue','opacity': 0.6})

#Next, we define how the protein will be shown in py3Dmol
#Note that we are also adding a style representation for active site residues
def viewprot(object,prot_PDBfile,resids):
  mol1 = open(prot_PDBfile, 'r').read()
  object.addModel(mol1,'pdb')
  object.setStyle({'cartoon': {'color':'spectrum'}})
  object.addStyle({'resi':resids},{'stick':{'colorscheme':'greenCarbon'}})

#Lastly, we combine the box grid and protein into a single viewer
def viewprotgrid(prot_PDBfile,resids,bxi,byi,bzi,bxf=10,byf=10,bzf=10):
  mol_view = py3Dmol.view(1000,1000,viewergrid=(1,2))
  definegrid(mol_view,bxi,byi,bzi,bxf,byf,bzf)
  viewprot(mol_view,prot_PDBfile,resids)
  mol_view.setBackgroundColor('0xffffff')
  mol_view.rotate(90, {'x':0,'y':1,'z':0},viewer=(0,1));
  mol_view.zoomTo()
  mol_view.show()


2. Ahora, utilizaremos nuestro ViewProtGrid para visualizar la proteína, los residuos del sitio de unión y una caja de cuadrícula de tamaño y posición variables que podemos manipular mediante un control deslizante a través de *ipywidgets*. Debes editar este visor indicando la ubicación del archivo PDB en la variable *prot_PDBfile* (por ejemplo, singlepath/'1hsg_prot.pdb') y los residuos que deseas mostrar del PDB en la variable *resids*.

Ejemplos de cómo usar la variable *prot_PDBfile*:

> prot_PDBfile = ['1hsg_prot.pdb'] (si el archivo PDB está en la ruta actual)

> prot_PDBfile = [singlepath/'1hsg_prot.pdb'] (si el archivo PDB está en una ruta definida como singlepath)

Ejemplos de cómo usar la variable *resids*:

> resids = [82] (muestra un solo residuo en la posición 82)

> resids = [82, 83, 84] (muestra los residuos 82, 83 o 84 por separado, los cuales puedes seleccionar en el visor)

> resids = [(82, 83, 84)] (muestra los residuos 82, 83 y 84 en la misma visualización)

> resids = ['82-84'] (muestra el rango de residuos 82-84 en la misma visualización)

**NOTA:** Este código falla al intentar mostrar dos residuos no consecutivos en la misma visualización.


In [ ]:
#@title Cargando el visor de Docking Box para un archivo PDB definido por el usuario y residuos resaltados

from ipywidgets import interact,fixed,IntSlider
import ipywidgets

pdbfile = "" # @param {type:"string"}
resids = None # @param {type:"raw"}
interact(viewprotgrid,
#ADD YOUR PDB LOCATION AND FILENAME HERE
         prot_PDBfile = pdbfile,
#ADD THE RESIDUES YOU WANT TO VISUALIZE HERE
         resids = resids,
         bxi=ipywidgets.IntSlider(min=-100,max=100, step=1),
         byi=ipywidgets.IntSlider(min=-100,max=100, step=1),
         bzi=ipywidgets.IntSlider(min=-100,max=100, step=1),
         bxf=ipywidgets.IntSlider(min=4,max=40, step=1),
         byf=ipywidgets.IntSlider(min=4,max=30, step=1),
         bzf=ipywidgets.IntSlider(min=4,max=30, step=1))

## 3. Ahora, generaremos un archivo de configuración para **Autodock Vina**. Como era de esperar, el archivo de configuración contiene información sobre la proteína objetivo y el ligando, así como la posición y dimensiones de la caja de cuadrícula que define el espacio de búsqueda.

Para definir la caja de cuadrícula, utilizarás las coordenadas de origen y tamaño que definiste manualmente en el paso anterior.

El siguiente es un ejemplo de un archivo estándar de configuración de **Autodock Vina**, que incluye varias variables editables. Una variable muy relevante es la **exhaustividad**, es decir, el número de ejecuciones independientes a partir de conformaciones aleatorias, y por lo tanto, la cantidad de esfuerzo computacional durante el acoplamiento molecular.

```plaintext
# ARCHIVO DE CONFIGURACIÓN

# OPCIONES DE ENTRADA
receptor = [archivo pdbqt de la proteína objetivo]
ligand = [archivo pdbqt del ligando]
flex = [residuos flexibles en el receptor en formato pdbqt]

# CONFIGURACIONES DEL ESPACIO DE BÚSQUEDA
# Centro de la caja (coordenadas x, y y z)
center_x = [valor]
center_y = [valor]
center_z = [valor]
# Tamaño de la caja (dimensiones en x, y y z)
size_x = [valor]
size_y = [valor]
size_z = [valor]

# OPCIONES DE SALIDA
# out = [archivo pdbqt de salida para todas las conformaciones]
# log = [archivo de registro de salida para las energías de unión]

# OTRAS OPCIONES
cpu = [valor] # más CPUs reducen el tiempo de cálculo
exhaustiveness = [valor] # tiempo de búsqueda para encontrar el mínimo global, el valor predeterminado es 8
num_modes = [valor] # número máximo de modos de unión para generar, el valor predeterminado es 9
energy_range = [valor] # diferencia máxima de energía entre el mejor modo de unión y el peor mostrado (kcal/mol), el valor predeterminado es 3
seed = [valor] # semilla aleatoria explícita, no es obligatoria
```
El siguiente script creará este archivo para nuestro procedimiento de acoplamiento. Debes agregar la posición y dimensiones de tu caja de cuadrícula.

In [ ]:
#@title Generating an AutoDock Vina file
receptor = "" # @param {type:"string"}
ligand = "" # @param {type:"string"}
center_x = "" # @param {type:"string"}
center_y = "" # @param {type:"string"}
center_z = "" # @param {type:"string"}
size_x = "" # @param {type:"string"}
size_y = "" # @param {type:"string"}
size_z = "" # @param {type:"string"}
with open(singlepath / "config_singledock","w") as f:
  f.write("#CONFIGURATION FILE (options not used are commented) \n")
  f.write("\n")
  f.write("#INPUT OPTIONS \n")
  f.write("receptor = " + receptor +"\n")
  f.write("ligand = " + ligand +"\n")
  f.write("#flex = [flexible residues in receptor in pdbqt format] \n")
  f.write("#SEARCH SPACE CONFIGURATIONS \n")
  f.write("#Center of the box (values bxi, byi and bzi) \n")
#CHANGE THE FOLLOWING DATA WITH YOUR BOX CENTER COORDINATES
  f.write("center_x = " + center_x + "\n")
  f.write("center_y = " + center_y + "\n")
  f.write("center_z = " + center_z + "\n")
#CHANGE THE FOLLOWING DATA WITH YOUR BOX DIMENSIONS
  f.write("#Size of the box (values bxf, byf and bzf) \n")
  f.write("size_x = " + size_x + "\n")
  f.write("size_y = " + size_y + "\n")
  f.write("size_z = " + size_z + "\n")
  f.write("#OUTPUT OPTIONS \n")
  f.write("#out = \n")
  f.write("#log = \n")
  f.write("\n")
  f.write("#OTHER OPTIONS \n")
  f.write("#cpu =  \n")
  f.write("#exhaustiveness = \n")
  f.write("#num_modes = \n")
  f.write("#energy_range = \n")
  f.write("#seed = ")

4. Por último, entraremos en la carpeta que creamos para el experimento de acoplamiento y **realizaremos nuestro primer acoplamiento molecular con Autodock Vina**.

  Una vez que ejecutes las líneas de código que se muestran a continuación, Autodock te mostrará una barra de progreso (si todo va según lo esperado). **Esta simulación no debería tomar más de 5 minutos**.
  
  Ten en cuenta que estamos definiendo los nombres de archivo de salida y el archivo de registro fuera del archivo de configuración.


In [ ]:
#Cambiando al directorio de acoplamiento único
os.chdir(singlepath)
#Ejecutando Autodock Vina con nuestro archivo de configuración
%vina --config config_singledock --out output.pdbqt | tee output.log
#Saliendo del directorio
os.chdir("/content/")

5. Una vez que haya finalizado la ejecución del acoplamiento molecular, compararemos las poses de acoplamiento con la pose resuelta experimentalmente para el indinavir. De hecho, la estructura de la proteasa del VIH-2 que descargaste al principio de este tutorial se resolvió con indinavir unido a ella.

  Las siguientes líneas de código son similares a lo que hicimos al extraer las líneas 'ATOM' del archivo PDB, pero ahora estamos extrayendo las líneas que contienen **'MK1'**, el nombre del ligando en este archivo PDB.


In [ ]:
# Aquí, extraeremos Indinavir, que está presente en la estructura de
# la proteasa del VIH-2 (¡sí! ¡esto es una simulación con validación experimental!)
# El enfoque es similar a imprimir las líneas ATOM y TER, pero estamos utilizando
# el nombre de residuo dado al ligando en la estructura resuelta experimentalmente: MK1
protein = "1hsg.pdb"

with open(singlepath/"xtal_ligand.pdb","w") as g:
  f = open(protein,'r')
  for line in f:
    row = line.split()
    if "MK1" in row:
      g.write(line)
  g.write("END")

6. También necesitamos las diferentes poses de acoplamiento generadas como resultado de la simulación de acoplamiento molecular. Separaremos estas poses en archivos PDB individuales utilizando **babel**, comenzando con el archivo numerado como 1 correspondiente a la pose de menor energía.


In [ ]:
# Necesitamos convertir nuestros resultados de Autodock Vina de pdbqt a pdb
# Para esto, usaremos babel
!obabel -ipdbqt $singlepath/output.pdbqt -opdb -O $singlepath/indinavir_dock.pdb -m

7. Finalmente, cargaremos nuestra proteína objetivo en **py3Dmol**, junto con cualquier pose de acoplamiento de nuestra elección y la pose de unión de indinavir resuelta experimentalmente para su comparación.


In [ ]:
view = py3Dmol.view()
view.setBackgroundColor('white')

# Cargando la proteína objetivo como archivo PDB
view.addModel(open(singlepath/'1hsg_prot.pdb', 'r').read(), 'pdb')
view.setStyle({'cartoon': {'color':'spectrum'}})
view.zoomTo()

# Cargando la pose de acoplamiento
view.addModel(open(singlepath/'indinavir_dock1.pdb', 'r').read(), 'pdb')
view.setStyle({'model': 1}, {'stick': {'colorscheme': 'greenCarbon'}})

# Cargando el modo de unión resuelto experimentalmente
view.addModel(open(singlepath/'xtal_ligand.pdb', 'r').read(), 'pdb')
view.setStyle({'resn':'MK1'}, {'stick':{}})
view.show()


¿Qué tan similar es tu pose de acoplamiento en comparación con la resuelta experimentalmente?

**📚TAREA:** Recuerda que el redocking, es decir, una simulación de acoplamiento molecular en la que el ligando unido al objetivo se utiliza como conformación inicial para el procedimiento de acoplamiento, se utiliza comúnmente como control. Con base en esta información, planifica una simulación de control utilizando el ligando que acabas de extraer en este tutorial.

>```python
># Ejemplo de preparación de la pose del ligando resuelta experimentalmente para redocking
>
>os.chdir(singlepath)
>!pythonsh /usr/local/bin/prepare_ligand4.py -l xtal_ligand.pdb -o $singlepath/xtal_ind.pdbqt -A hydrogens -U nphs_lps -v
>
># Luego, puedes usar esencialmente los mismos archivos de configuración,
>```



**¡Y esto es el final del sexto tutorial!** Si deseas descargar tus resultados, puedes comprimirlos en un archivo zip para su descarga manual.


In [ ]:
!zip -r singledocking.zip $singlepath
# Por defecto, la descarga automática está habilitada a través de las siguientes líneas
# pero debes desactivar tu bloqueador de anuncios para que funcione
from google.colab import files
files.download("/content/singledocking.zip")